In [1]:
import sys
sys.path.append(r'..\scripts')

import numpy as np

## Kernel

### Compute Kernel if not cached

In [ ]:
from compute_kernels import parallel_Li

In [ ]:
#Define simulation parameters

N = 120 # Number of spatial points
zb, zt = 1, 5 #bounds
k = 2 #wave number
alpha = 4.8
dz = (zt - zb)/N

#Radius function
R = lambda z : 1
R = np.vectorize(R)

if __name__ == '__main__':
    zg, oc, L = parallel_Li(zb, zt, N, k, R, num_workers = None)

In [ ]:
np.savez(f'../data_cache/discretizationN={N}.npz', zg=zg, oc=oc, L=L)

### Load Cached Kernel

In [2]:
#Define simulation parameters

N = 120 # Number of spatial points
zb, zt = 1, 5 #bounds
k = 2 #wave number
alpha = 4.8
dz = (zt - zb)/N

#Radius function
R = lambda z : 1
R = np.vectorize(R)

discretization = np.load(f'../data_cache/discretizationN={N}.npz')
zg, oc, Li = discretization['zg'], discretization['oc'], discretization['L']

## Series Exploration

In [17]:
from steady_state_analysis import compute_adomian_decomp_terms
from simulation import simulate
from dynamical_system import F_k
import matplotlib.pyplot as plt

In [9]:
#Default Parameters
T_final = 300
gamma = 0.1
S_hat = 100
delta_hat = 0.1
kappa = 30
Delta0_func = lambda z : 1
eta0_func = lambda z : 1 + 0j
Delta_E = np.vectorize(lambda z : 0.5)

sol_T, sol_Delta, sol_eta = simulate(kappa, gamma, S_hat, delta_hat, Delta0_func, eta0_func, Delta_E, R, k, zb, zt, N, T_final, Li, oc, solver='BDF')

In [14]:
Delta_invest = sol_Delta[:, -1]
lambd = 1/((oc @ Delta_invest) * dz - delta_hat - 1/(1j * S_hat)) 

terms, lambd = compute_adomian_decomp_terms(Delta_invest, Li, oc, dz, F_k(R, zg, k), S_hat, delta_hat, 110)

In [15]:
terms_full = []
for i, term in enumerate(terms):
    terms_full.append(lambd**(i+1) * term)
terms_full = np.array(terms_full)

In [ ]:
plt.plot(zg, lambd)

In [ ]:
plt.plot(zg, terms[0:10].T)